In [ ]:
# https://ourworldindata.org/income-inequality#how-are-the-incomes-of-the-rich-changing-relative-to-the-incomes-of-the-poor

using CSV
using DataFrames
using Query
using Gadfly

In [ ]:
household_income_raw = CSV.read("./data/real-disposable-household-income-indexed.csv")

In [ ]:
household_income = household_income_raw |>
    @rename(:Entity => :Country) |>
    DataFrame |>
    (df -> stack(df, r"\d+")) |>
    (df -> categorical(df, [:Country, :Code])) |>
    @rename(:variable => :Decile, :value => :Index) |>
    DataFrame

In [ ]:
set_default_plot_size(800px, 600px)

household_income |>
    @filter(_.Country == "United Kingdom") |>
    DataFrame |>
    (df -> plot(df, x=:Year,
                    y=:Index,
                    color=:Decile,
                    layer(Geom.line, Geom.point),
                    layer(yintercept=[100], Geom.hline(color=["black"], style=[:dot])),
                    Guide.title("Growth of Real Disposable Household Income by Decile")))

In [ ]:
countries = ["United Kingdom", "United States"]
household_income |>
    @filter(_.Country in countries) |>
    DataFrame |>
    (df -> plot(df, x=:Year,
                    y=:Index,
                    color=:Decile,
                    xgroup=:Country,
                    Geom.subplot_grid(layer(Geom.line, Geom.point),
                                      layer(DataFrame(yint=[100, 100], Country=countries),
                                            xgroup=:Country, yintercept=:yint,
                                            Geom.hline(color="black", style=:dot))),
                    Guide.title("Growth of Real Disposable Household Income by Decile")))